# Data Collector
- Meeting Audio Data Collector 
- Connects to ESP32 and records labeled audio data for ML training
- Usage: python data_collector.ipynb

In [1]:
import websocket
import json
import pandas as pd
import numpy as np
import threading
import time
import os
import sys
from datetime import datetime
from collections import deque
import signal

In [ ]:
class MeetingDataCollector:
    def __init__(self, esp32_ip="192.168.1.100", port=81):
        self.esp32_ip = esp32_ip
        self.port = port
        self.ws_url = f"ws://{esp32_ip}:{port}"
        
        # Data storage
        self.audio_data = deque(maxlen=10000)  # Store last 10k samples
        self.session_data = []
        self.recording = False
        self.connected = False
        
        # Session metadata
        self.session_id = None
        self.session_labels = {}
        self.start_time = None
        
        # Create directories
        os.makedirs("recordings", exist_ok=True)
        os.makedirs("labels", exist_ok=True)
        
        # WebSocket setup
        self.ws = None
        self.running = True
        
        # Setup signal handler for graceful shutdown
        signal.signal(signal.SIGINT, self.signal_handler)
    
    def signal_handler(self, signum, frame):
        """Handle Ctrl+C gracefully"""
        print("\nShutting down gracefully...")
        self.stop_recording()
        self.running = False
        if self.ws:
            self.ws.close()
        sys.exit(0)
    
    def on_message(self, ws, message):
        """Handle incoming WebSocket messages"""
        try:
            data = json.loads(message)
            
            # Add local timestamp
            data['local_timestamp'] = time.time()
            data['local_datetime'] = datetime.now().isoformat()
            
            # Store data
            self.audio_data.append(data)
            
            # If recording, add to session data
            if self.recording:
                self.session_data.append(data)
                
        except json.JSONDecodeError:
            print(f"Error parsing message: {message}")
        except Exception as e:
            print(f"Error in on_message: {e}")
    
    def on_error(self, ws, error):
        """Handle WebSocket errors"""
        print(f"WebSocket error: {error}")
        self.connected = False
    
    def on_close(self, ws, close_status_code, close_msg):
        """Handle WebSocket close"""
        print("WebSocket connection closed")
        self.connected = False
    
    def on_open(self, ws):
        """Handle WebSocket open"""
        print("Connected to ESP32!")
        self.connected = True
    
    def connect_to_esp32(self):
        """Connect to ESP32 WebSocket"""
        print(f"Connecting to ESP32 at {self.ws_url}...")
        
        try:
            self.ws = websocket.WebSocketApp(
                self.ws_url,
                on_message=self.on_message,
                on_error=self.on_error,
                on_close=self.on_close,
                on_open=self.on_open
            )
            
            # Start WebSocket in separate thread
            self.ws_thread = threading.Thread(target=self.ws.run_forever)
            self.ws_thread.daemon = True
            self.ws_thread.start()
            
            # Wait for connection
            timeout = 10
            for i in range(timeout):
                if self.connected:
                    return True
                time.sleep(1)
                print(f"Waiting for connection... ({i+1}/{timeout})")
            
            print("Connection timeout!")
            return False
            
        except Exception as e:
            print(f"Connection failed: {e}")
            return False
    
    def start_recording(self):
        """Start a new recording session"""
        if self.recording:
            print("Already recording!")
            return
        
        # Generate session ID
        self.session_id = f"meeting_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        self.start_time = time.time()
        self.session_data = []
        self.session_labels = {
            'session_id': self.session_id,
            'start_time': datetime.now().isoformat(),
            'esp32_ip': self.esp32_ip
        }
        
        self.recording = True
        print(f"Recording started: {self.session_id}")
    
    def stop_recording(self):
        """Stop recording and save data"""
        if not self.recording:
            print("Not currently recording!")
            return
        
        self.recording = False
        duration = time.time() - self.start_time
        
        print(f"Recording stopped. Duration: {duration:.1f} seconds")
        print(f"Samples collected: {len(self.session_data)}")
        
        if len(self.session_data) > 0:
            self.save_session()
        else:
            print("No data to save!")
    
    def save_session(self):
        """Save session data to files"""
        if not self.session_data:
            print("No data to save!")
            return
        
        try:
            # Add final metadata
            self.session_labels['end_time'] = datetime.now().isoformat()
            self.session_labels['duration_seconds'] = time.time() - self.start_time
            self.session_labels['sample_count'] = len(self.session_data)
            
            # Save audio data as JSON
            audio_filename = f"recordings/{self.session_id}_audio.json"
            with open(audio_filename, 'w') as f:
                json.dump(self.session_data, f, indent=2)
            
            # Save labels as JSON
            labels_filename = f"labels/{self.session_id}_labels.json"
            with open(labels_filename, 'w') as f:
                json.dump(self.session_labels, f, indent=2)
            
            # Append to master CSV
            self.update_master_csv()
            
            print(f"Session saved:")
            print(f"   Audio: {audio_filename}")
            print(f"   Labels: {labels_filename}")
            
        except Exception as e:
            print(f"Error saving session: {e}")
    
    def update_master_csv(self):
        """Update master CSV with session summary"""
        csv_file = "labels/sessions_master.csv"
        
        # Create summary row
        summary = {
            'session_id': self.session_labels['session_id'],
            'start_time': self.session_labels['start_time'],
            'duration_seconds': self.session_labels['duration_seconds'],
            'sample_count': self.session_labels['sample_count'],
            'speaker_count': self.session_labels.get('speaker_count', 'unknown'),
            'meeting_type': self.session_labels.get('meeting_type', 'unknown'),
            'energy_level': self.session_labels.get('energy_level', 'unknown'),
            'background_noise': self.session_labels.get('background_noise', 'unknown'),
            'notes': self.session_labels.get('notes', '')
        }
        
        # Load existing CSV or create new one
        if os.path.exists(csv_file):
            df = pd.read_csv(csv_file)
            df = pd.concat([df, pd.DataFrame([summary])], ignore_index=True)
        else:
            df = pd.DataFrame([summary])
        
        df.to_csv(csv_file, index=False)
        print(f"Updated master CSV: {csv_file}")
    
    def add_label(self, key, value):
        """Add a label to current session"""
        if not self.recording:
            print("Start recording first!")
            return
        
        self.session_labels[key] = value
        print(f"Added label: {key} = {value}")
    
    def show_current_data(self):
        """Display current audio data"""
        if not self.audio_data:
            print("No data received yet...")
            return
        
        # Get last 5 samples
        recent_samples = list(self.audio_data)[-5:]
        
        print("Recent Audio Data:")
        for i, sample in enumerate(recent_samples, 1):
            left = sample.get('leftMic', 0)
            right = sample.get('rightMic', 0)
            diff = sample.get('difference', 0)
            avg = sample.get('averageLevel', 0)
            
            print(f"   {i}: L:{left:6.1f}dB | R:{right:6.1f}dB | Diff:{diff:6.1f}dB | Avg:{avg:6.1f}dB")
    
    def show_status(self):
        """Display current status"""
        status = "RECORDING" if self.recording else "STOPPED"
        connection = "CONNECTED" if self.connected else "DISCONNECTED"
        
        print(f"\n{'='*60}")
        print(f"Meeting Audio Recorder v1.0")
        print(f"ESP32: {self.esp32_ip}:{self.port} - {connection}")
        print(f"Status: {status}")
        
        if self.recording:
            duration = time.time() - self.start_time
            print(f"Recording Duration: {duration:.1f} seconds")
            print(f"Samples Collected: {len(self.session_data)}")
            print(f"Session ID: {self.session_id}")
            
            print(f"\nCurrent Labels:")
            for key, value in self.session_labels.items():
                if key not in ['session_id', 'start_time', 'esp32_ip']:
                    print(f"   {key}: {value}")
        
        print(f"{'='*60}")
    
    def interactive_menu(self):
        """Main interactive menu"""
        print("Starting Meeting Audio Data Collector...")
        
        # Connect to ESP32
        if not self.connect_to_esp32():
            print("Failed to connect to ESP32. Please check IP and ensure ESP32 is running.")
            return
        
        # Wait a moment for data to start flowing
        time.sleep(2)
        
        while self.running:
            try:
                self.show_status()
                self.show_current_data()
                
                print(f"\nCommands:")
                print(f"[s] Start/Stop recording")
                print(f"[l] Add label")
                print(f"[v] View recent data")
                print(f"[i] Change ESP32 IP")
                print(f"[q] Quit")
                
                choice = input("\nEnter command: ").strip().lower()
                
                if choice == 's':
                    if self.recording:
                        self.stop_recording()
                    else:
                        self.start_recording()
                
                elif choice == 'l':
                    if not self.recording:
                        print("Start recording first!")
                        continue
                    
                    print("\nAvailable labels:")
                    print("[1] Speaker count (1, 2, 3+)")
                    print("[2] Meeting type (discussion, presentation, brainstorm, argument)")
                    print("[3] Energy level (low, medium, high)")
                    print("[4] Background noise (none, low, medium, high)")
                    print("[5] Custom label")
                    
                    label_choice = input("Select label type (1-5): ").strip()
                    
                    if label_choice == '1':
                        value = input("Number of speakers (1, 2, 3+): ").strip()
                        self.add_label('speaker_count', value)
                    elif label_choice == '2':
                        value = input("Meeting type (discussion/presentation/brainstorm/argument): ").strip()
                        self.add_label('meeting_type', value)
                    elif label_choice == '3':
                        value = input("Energy level (low/medium/high): ").strip()
                        self.add_label('energy_level', value)
                    elif label_choice == '4':
                        value = input("Background noise (none/low/medium/high): ").strip()
                        self.add_label('background_noise', value)
                    elif label_choice == '5':
                        key = input("Label name: ").strip()
                        value = input("Label value: ").strip()
                        self.add_label(key, value)
                
                elif choice == 'v':
                    self.show_current_data()
                    input("Press Enter to continue...")
                
                elif choice == 'i':
                    new_ip = input(f"Current IP: {self.esp32_ip}. Enter new IP: ").strip()
                    if new_ip:
                        self.esp32_ip = new_ip
                        self.ws_url = f"ws://{new_ip}:{self.port}"
                        print(f"IP updated to {new_ip}. Restart to connect.")
                
                elif choice == 'q':
                    if self.recording:
                        print("Still recording! Stopping...")
                        self.stop_recording()
                    break
                
                else:
                    print("Invalid command!")
                
            except KeyboardInterrupt:
                break
            except Exception as e:
                print(f"Error in menu: {e}")
        
        print("Goodbye!")

def main():
    """Main function"""
    print("Meeting Audio Data Collector v1.0")
    print("="*50)
    
    # Get ESP32 IP
    default_ip = "192.168.1.100"
    esp32_ip = input(f"ESP32 IP address ({default_ip}): ").strip()
    if not esp32_ip:
        esp32_ip = default_ip
    
    # Create collector and start
    collector = MeetingDataCollector(esp32_ip)
    collector.interactive_menu()

if __name__ == "__main__":
    main()

Meeting Audio Data Collector v1.0


ESP32 IP address (192.168.1.100):  192.168.0.110


Starting Meeting Audio Data Collector...
Connecting to ESP32 at ws://192.168.0.110:81...
Connected to ESP32!
Waiting for connection... (1/10)

Meeting Audio Recorder v1.0
ESP32: 192.168.0.110:81 - CONNECTED
Status: STOPPED
Recent Audio Data:
   1: L:  66.0dB | R:  64.7dB | Diff:   1.3dB | Avg:  65.4dB
   2: L:  59.2dB | R:  57.9dB | Diff:   1.3dB | Avg:  58.5dB
   3: L:  60.6dB | R:  60.0dB | Diff:   0.6dB | Avg:  60.3dB
   4: L:  58.5dB | R:  58.8dB | Diff:  -0.4dB | Avg:  58.6dB

Commands:
[s] Start/Stop recording
[l] Add label
[v] View recent data
[i] Change ESP32 IP
[q] Quit



Enter command:  s


Recording started: meeting_20250617_144307

Meeting Audio Recorder v1.0
ESP32: 192.168.0.110:81 - CONNECTED
Status: RECORDING
Recording Duration: 0.0 seconds
Samples Collected: 0
Session ID: meeting_20250617_144307

Current Labels:
Recent Audio Data:
   1: L:  83.9dB | R:  77.1dB | Diff:   6.8dB | Avg:  80.5dB
   2: L:  91.9dB | R:  88.2dB | Diff:   3.8dB | Avg:  90.0dB
   3: L:  61.0dB | R:  59.0dB | Diff:   2.0dB | Avg:  60.0dB
   4: L:  61.9dB | R:  67.3dB | Diff:  -5.5dB | Avg:  64.6dB
   5: L:  66.6dB | R:  61.8dB | Diff:   4.8dB | Avg:  64.2dB

Commands:
[s] Start/Stop recording
[l] Add label
[v] View recent data
[i] Change ESP32 IP
[q] Quit


...


Available labels:
[1] Speaker count (1, 2, 3+)
[2] Meeting type (discussion, presentation, brainstorm, argument)
[3] Energy level (low, medium, high)
[4] Background noise (none, low, medium, high)
[5] Custom label


Select label type (1-5):  4
Background noise (none/low/medium/high):  low


Added label: background_noise = low

Meeting Audio Recorder v1.0
ESP32: 192.168.0.110:81 - CONNECTED
Status: RECORDING
Recording Duration: 271.0 seconds
Samples Collected: 476
Session ID: meeting_20250617_145139

Current Labels:
   meeting_type: discussion
   background_noise: low
Recent Audio Data:
   1: L:  59.2dB | R:  60.6dB | Diff:  -1.4dB | Avg:  59.9dB
   2: L:  62.9dB | R:  56.4dB | Diff:   6.5dB | Avg:  59.7dB
   3: L:  61.9dB | R:  58.2dB | Diff:   3.7dB | Avg:  60.1dB
   4: L:  65.0dB | R:  68.6dB | Diff:  -3.5dB | Avg:  66.8dB
   5: L:  66.2dB | R:  61.1dB | Diff:   5.1dB | Avg:  63.7dB

Commands:
[s] Start/Stop recording
[l] Add label
[v] View recent data
[i] Change ESP32 IP
[q] Quit



Enter command:  s


Recording stopped. Duration: 377.1 seconds
Samples collected: 650
Updated master CSV: labels/sessions_master.csv
Session saved:
   Audio: recordings/meeting_20250617_145139_audio.json
   Labels: labels/meeting_20250617_145139_labels.json

Meeting Audio Recorder v1.0
ESP32: 192.168.0.110:81 - CONNECTED
Status: STOPPED
Recent Audio Data:
   1: L:  59.0dB | R:  67.2dB | Diff:  -8.1dB | Avg:  63.1dB
   2: L:  66.4dB | R:  68.3dB | Diff:  -1.9dB | Avg:  67.4dB
   3: L:  58.8dB | R:  57.4dB | Diff:   1.4dB | Avg:  58.1dB
   4: L:  67.3dB | R:  63.3dB | Diff:   4.0dB | Avg:  65.3dB
   5: L:  63.7dB | R:  60.0dB | Diff:   3.7dB | Avg:  61.9dB

Commands:
[s] Start/Stop recording
[l] Add label
[v] View recent data
[i] Change ESP32 IP
[q] Quit


In [2]:
# 🎤 Meeting Audio Data Collector v1.0
# ESP32 IP address (192.168.1.100): [Enter your ESP32 IP]

# 🔌 Connecting to ESP32 at ws://192.168.1.100:81...
# ✅ Connected to ESP32!

# ====================================================
# 🎤 Meeting Audio Recorder v1.0
# 📡 ESP32: 192.168.1.100:81 - 🟢 CONNECTED
# 📹 Status: ⏸️  STOPPED

# 📊 Recent Audio Data:
#    1: L: 45.2dB | R: 42.1dB | Diff:  3.1dB | Avg: 43.7dB
#    2: L: 46.1dB | R: 41.8dB | Diff:  4.3dB | Avg: 44.0dB
#    ...

# 📋 Commands:
# [s] Start/Stop recording
# [l] Add label  
# [v] View recent data
# [i] Change ESP32 IP
# [q] Quit

# 👉 Enter command: s

Meeting Audio Data Collector v1.0


ESP32 IP address (192.168.1.100):  192.168.0.110


Starting Meeting Audio Data Collector...
Connecting to ESP32 at ws://192.168.0.110:81...
Connected to ESP32!
Waiting for connection... (1/10)

Meeting Audio Recorder v1.0
ESP32: 192.168.0.110:81 - CONNECTED
Status: STOPPED
Recent Audio Data:
   1: L:  77.5dB | R:  73.2dB | Diff:   4.3dB | Avg:  75.3dB
   2: L:  66.0dB | R:  56.3dB | Diff:   9.7dB | Avg:  61.1dB
   3: L:  62.9dB | R:  60.7dB | Diff:   2.2dB | Avg:  61.8dB
   4: L:  67.4dB | R:  66.5dB | Diff:   0.8dB | Avg:  66.9dB

Commands:
[s] Start/Stop recording
[l] Add label
[v] View recent data
[i] Change ESP32 IP
[q] Quit



Enter command:  s


Recording started: meeting_20250620_130013

Meeting Audio Recorder v1.0
ESP32: 192.168.0.110:81 - CONNECTED
Status: RECORDING
Recording Duration: 0.0 seconds
Samples Collected: 0
Session ID: meeting_20250620_130013

Current Labels:
Recent Audio Data:
   1: L:  62.9dB | R:  60.7dB | Diff:   2.2dB | Avg:  61.8dB
   2: L:  67.4dB | R:  66.5dB | Diff:   0.8dB | Avg:  66.9dB
   3: L:  64.7dB | R:  63.1dB | Diff:   1.7dB | Avg:  63.9dB
   4: L:  67.7dB | R:  65.8dB | Diff:   1.9dB | Avg:  66.8dB
   5: L:  67.3dB | R:  65.2dB | Diff:   2.0dB | Avg:  66.3dB

Commands:
[s] Start/Stop recording
[l] Add label
[v] View recent data
[i] Change ESP32 IP
[q] Quit


...


Available labels:
[1] Speaker count (1, 2, 3+)
[2] Meeting type (discussion, presentation, brainstorm, argument)
[3] Energy level (low, medium, high)
[4] Background noise (none, low, medium, high)
[5] Custom label


Select label type (1-5):  4
Background noise (none/low/medium/high):  high


Added label: background_noise = high

Meeting Audio Recorder v1.0
ESP32: 192.168.0.110:81 - CONNECTED
Status: RECORDING
Recording Duration: 65.3 seconds
Samples Collected: 114
Session ID: meeting_20250620_130618

Current Labels:
   speaker_count: 3
   meeting_type: discussion
   energy_level: high
   background_noise: high
Recent Audio Data:
   1: L:  62.3dB | R:  59.1dB | Diff:   3.3dB | Avg:  60.7dB
   2: L:  65.7dB | R:  55.8dB | Diff:   9.9dB | Avg:  60.7dB
   3: L:  60.8dB | R:  58.8dB | Diff:   2.0dB | Avg:  59.8dB
   4: L:  61.4dB | R:  58.5dB | Diff:   2.9dB | Avg:  59.9dB
   5: L:  61.1dB | R:  61.0dB | Diff:   0.1dB | Avg:  61.1dB

Commands:
[s] Start/Stop recording
[l] Add label
[v] View recent data
[i] Change ESP32 IP
[q] Quit



Enter command:  s


Recording stopped. Duration: 304.2 seconds
Samples collected: 534
Updated master CSV: labels/sessions_master.csv
Session saved:
   Audio: recordings/meeting_20250620_130618_audio.json
   Labels: labels/meeting_20250620_130618_labels.json

Meeting Audio Recorder v1.0
ESP32: 192.168.0.110:81 - CONNECTED
Status: STOPPED
Recent Audio Data:
   1: L:  72.6dB | R:  68.8dB | Diff:   3.7dB | Avg:  70.7dB
   2: L:  70.9dB | R:  66.3dB | Diff:   4.6dB | Avg:  68.6dB
   3: L:  70.9dB | R:  66.5dB | Diff:   4.4dB | Avg:  68.7dB
   4: L:  71.8dB | R:  68.8dB | Diff:   3.0dB | Avg:  70.3dB
   5: L:  70.6dB | R:  69.9dB | Diff:   0.7dB | Avg:  70.3dB

Commands:
[s] Start/Stop recording
[l] Add label
[v] View recent data
[i] Change ESP32 IP
[q] Quit


In [ ]:
class MeetingDataCollector:
    def __init__(self, esp32_ip="192.168.1.100", port=81):
        self.esp32_ip = esp32_ip
        self.port = port
        self.ws_url = f"ws://{esp32_ip}:{port}"
        
        # Data storage
        self.audio_data = deque(maxlen=10000)  # Store last 10k samples
        self.session_data = []
        self.recording = False
        self.connected = False
        
        # Session metadata
        self.session_id = None
        self.session_labels = {}
        self.start_time = None
        
        # Create directories
        os.makedirs("recordings", exist_ok=True)
        os.makedirs("labels", exist_ok=True)
        
        # WebSocket setup
        self.ws = None
        self.running = True
        
        # Setup signal handler for graceful shutdown
        signal.signal(signal.SIGINT, self.signal_handler)
    
    def signal_handler(self, signum, frame):
        """Handle Ctrl+C gracefully"""
        print("\nShutting down gracefully...")
        self.stop_recording()
        self.running = False
        if self.ws:
            self.ws.close()
        sys.exit(0)
    
    def on_message(self, ws, message):
        """Handle incoming WebSocket messages"""
        try:
            data = json.loads(message)
            
            # Add local timestamp
            data['local_timestamp'] = time.time()
            data['local_datetime'] = datetime.now().isoformat()
            
            # Store data
            self.audio_data.append(data)
            
            # If recording, add to session data
            if self.recording:
                self.session_data.append(data)
                
        except json.JSONDecodeError:
            print(f"Error parsing message: {message}")
        except Exception as e:
            print(f"Error in on_message: {e}")
    
    def on_error(self, ws, error):
        """Handle WebSocket errors"""
        print(f"WebSocket error: {error}")
        self.connected = False
    
    def on_close(self, ws, close_status_code, close_msg):
        """Handle WebSocket close"""
        print("WebSocket connection closed")
        self.connected = False
    
    def on_open(self, ws):
        """Handle WebSocket open"""
        print("Connected to ESP32!")
        self.connected = True
    
    def connect_to_esp32(self):
        """Connect to ESP32 WebSocket"""
        print(f"Connecting to ESP32 at {self.ws_url}...")
        
        try:
            self.ws = websocket.WebSocketApp(
                self.ws_url,
                on_message=self.on_message,
                on_error=self.on_error,
                on_close=self.on_close,
                on_open=self.on_open
            )
            
            # Start WebSocket in separate thread
            self.ws_thread = threading.Thread(target=self.ws.run_forever)
            self.ws_thread.daemon = True
            self.ws_thread.start()
            
            # Wait for connection
            timeout = 10
            for i in range(timeout):
                if self.connected:
                    return True
                time.sleep(1)
                print(f"Waiting for connection... ({i+1}/{timeout})")
            
            print("Connection timeout!")
            return False
            
        except Exception as e:
            print(f"Connection failed: {e}")
            return False
    
    def start_recording(self):
        """Start a new recording session"""
        if self.recording:
            print("Already recording!")
            return
        
        # Generate session ID
        self.session_id = f"meeting_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        self.start_time = time.time()
        self.session_data = []
        self.session_labels = {
            'session_id': self.session_id,
            'start_time': datetime.now().isoformat(),
            'esp32_ip': self.esp32_ip
        }
        
        self.recording = True
        print(f"Recording started: {self.session_id}")
    
    def stop_recording(self):
        """Stop recording and save data"""
        if not self.recording:
            print("Not currently recording!")
            return
        
        self.recording = False
        duration = time.time() - self.start_time
        
        print(f"Recording stopped. Duration: {duration:.1f} seconds")
        print(f"Samples collected: {len(self.session_data)}")
        
        if len(self.session_data) > 0:
            self.save_session()
        else:
            print("No data to save!")
    
    def save_session(self):
        """Save session data to files"""
        if not self.session_data:
            print("No data to save!")
            return
        
        try:
            # Add final metadata
            self.session_labels['end_time'] = datetime.now().isoformat()
            self.session_labels['duration_seconds'] = time.time() - self.start_time
            self.session_labels['sample_count'] = len(self.session_data)
            
            # Save audio data as JSON
            audio_filename = f"recordings/{self.session_id}_audio.json"
            with open(audio_filename, 'w') as f:
                json.dump(self.session_data, f, indent=2)
            
            # Save labels as JSON
            labels_filename = f"labels/{self.session_id}_labels.json"
            with open(labels_filename, 'w') as f:
                json.dump(self.session_labels, f, indent=2)
            
            # Append to master CSV
            self.update_master_csv()
            
            print(f"Session saved:")
            print(f"   Audio: {audio_filename}")
            print(f"   Labels: {labels_filename}")
            
        except Exception as e:
            print(f"Error saving session: {e}")
    
    def update_master_csv(self):
        """Update master CSV with session summary"""
        csv_file = "labels/sessions_master.csv"
        
        # Create summary row
        summary = {
            'session_id': self.session_labels['session_id'],
            'start_time': self.session_labels['start_time'],
            'duration_seconds': self.session_labels['duration_seconds'],
            'sample_count': self.session_labels['sample_count'],
            'speaker_count': self.session_labels.get('speaker_count', 'unknown'),
            'meeting_type': self.session_labels.get('meeting_type', 'unknown'),
            'energy_level': self.session_labels.get('energy_level', 'unknown'),
            'background_noise': self.session_labels.get('background_noise', 'unknown'),
            'notes': self.session_labels.get('notes', '')
        }
        
        # Load existing CSV or create new one
        if os.path.exists(csv_file):
            df = pd.read_csv(csv_file)
            df = pd.concat([df, pd.DataFrame([summary])], ignore_index=True)
        else:
            df = pd.DataFrame([summary])
        
        df.to_csv(csv_file, index=False)
        print(f"Updated master CSV: {csv_file}")
    
    def add_label(self, key, value):
        """Add a label to current session"""
        if not self.recording:
            print("Start recording first!")
            return
        
        self.session_labels[key] = value
        print(f"Added label: {key} = {value}")
    
    def show_current_data(self):
        """Display current audio data"""
        if not self.audio_data:
            print("No data received yet...")
            return
        
        # Get last 5 samples
        recent_samples = list(self.audio_data)[-5:]
        
        print("Recent Audio Data:")
        for i, sample in enumerate(recent_samples, 1):
            left = sample.get('leftMic', 0)
            right = sample.get('rightMic', 0)
            diff = sample.get('difference', 0)
            avg = sample.get('averageLevel', 0)
            
            print(f"   {i}: L:{left:6.1f}dB | R:{right:6.1f}dB | Diff:{diff:6.1f}dB | Avg:{avg:6.1f}dB")
    
    def show_status(self):
        """Display current status"""
        status = "RECORDING" if self.recording else "STOPPED"
        connection = "CONNECTED" if self.connected else "DISCONNECTED"
        
        print(f"\n{'='*60}")
        print(f"Meeting Audio Recorder v1.0")
        print(f"ESP32: {self.esp32_ip}:{self.port} - {connection}")
        print(f"Status: {status}")
        
        if self.recording:
            duration = time.time() - self.start_time
            print(f"Recording Duration: {duration:.1f} seconds")
            print(f"Samples Collected: {len(self.session_data)}")
            print(f"Session ID: {self.session_id}")
            
            print(f"\nCurrent Labels:")
            for key, value in self.session_labels.items():
                if key not in ['session_id', 'start_time', 'esp32_ip']:
                    print(f"   {key}: {value}")
        
        print(f"{'='*60}")
    
    def interactive_menu(self):
        """Main interactive menu"""
        print("Starting Meeting Audio Data Collector...")
        
        # Connect to ESP32
        if not self.connect_to_esp32():
            print("Failed to connect to ESP32. Please check IP and ensure ESP32 is running.")
            return
        
        # Wait a moment for data to start flowing
        time.sleep(2)
        
        while self.running:
            try:
                self.show_status()
                self.show_current_data()
                
                print(f"\nCommands:")
                print(f"[s] Start/Stop recording")
                print(f"[l] Add label")
                print(f"[v] View recent data")
                print(f"[i] Change ESP32 IP")
                print(f"[q] Quit")
                
                choice = input("\nEnter command: ").strip().lower()
                
                if choice == 's':
                    if self.recording:
                        self.stop_recording()
                    else:
                        self.start_recording()
                
                elif choice == 'l':
                    if not self.recording:
                        print("Start recording first!")
                        continue
                    
                    print("\nAvailable labels:")
                    print("[1] Speaker count (1, 2, 3+)")
                    print("[2] Meeting type (discussion, presentation, brainstorm, argument)")
                    print("[3] Energy level (low, medium, high)")
                    print("[4] Background noise (none, low, medium, high)")
                    print("[5] Custom label")
                    
                    label_choice = input("Select label type (1-5): ").strip()
                    
                    if label_choice == '1':
                        value = input("Number of speakers (1, 2, 3+): ").strip()
                        self.add_label('speaker_count', value)
                    elif label_choice == '2':
                        value = input("Meeting type (discussion/presentation/brainstorm/argument): ").strip()
                        self.add_label('meeting_type', value)
                    elif label_choice == '3':
                        value = input("Energy level (low/medium/high): ").strip()
                        self.add_label('energy_level', value)
                    elif label_choice == '4':
                        value = input("Background noise (none/low/medium/high): ").strip()
                        self.add_label('background_noise', value)
                    elif label_choice == '5':
                        key = input("Label name: ").strip()
                        value = input("Label value: ").strip()
                        self.add_label(key, value)
                
                elif choice == 'v':
                    self.show_current_data()
                    input("Press Enter to continue...")
                
                elif choice == 'i':
                    new_ip = input(f"Current IP: {self.esp32_ip}. Enter new IP: ").strip()
                    if new_ip:
                        self.esp32_ip = new_ip
                        self.ws_url = f"ws://{new_ip}:{self.port}"
                        print(f"IP updated to {new_ip}. Restart to connect.")
                
                elif choice == 'q':
                    if self.recording:
                        print("Still recording! Stopping...")
                        self.stop_recording()
                    break
                
                else:
                    print("Invalid command!")
                
            except KeyboardInterrupt:
                break
            except Exception as e:
                print(f"Error in menu: {e}")
        
        print("Goodbye!")

def main():
    """Main function"""
    print("Meeting Audio Data Collector v1.0")
    print("="*50)
    
    # Get ESP32 IP
    default_ip = "192.168.1.100"
    esp32_ip = input(f"ESP32 IP address ({default_ip}): ").strip()
    if not esp32_ip:
        esp32_ip = default_ip
    
    # Create collector and start
    collector = MeetingDataCollector(esp32_ip)
    collector.interactive_menu()

if __name__ == "__main__":
    main()

Meeting Audio Data Collector v1.0


ESP32 IP address (192.168.1.100):  192.168.0.110


Starting Meeting Audio Data Collector...
Connecting to ESP32 at ws://192.168.0.110:81...
Waiting for connection... (1/10)
Connected to ESP32!
Waiting for connection... (2/10)

Meeting Audio Recorder v1.0
ESP32: 192.168.0.110:81 - CONNECTED
Status: STOPPED
Recent Audio Data:
   1: L:  68.2dB | R:  68.0dB | Diff:   0.2dB | Avg:  68.1dB
   2: L:  61.0dB | R:  66.6dB | Diff:  -5.5dB | Avg:  63.8dB
   3: L:  63.1dB | R:  62.9dB | Diff:   0.2dB | Avg:  63.0dB
   4: L:  59.9dB | R:  66.7dB | Diff:  -6.8dB | Avg:  63.3dB

Commands:
[s] Start/Stop recording
[l] Add label
[v] View recent data
[i] Change ESP32 IP
[q] Quit



Enter command:  s


Recording started: meeting_20250624_132317

Meeting Audio Recorder v1.0
ESP32: 192.168.0.110:81 - CONNECTED
Status: RECORDING
Recording Duration: 0.0 seconds
Samples Collected: 0
Session ID: meeting_20250624_132317

Current Labels:
Recent Audio Data:
   1: L:  66.9dB | R:  63.5dB | Diff:   3.5dB | Avg:  65.2dB
   2: L:  59.6dB | R:  63.3dB | Diff:  -3.7dB | Avg:  61.4dB
   3: L:  68.2dB | R:  64.9dB | Diff:   3.3dB | Avg:  66.5dB
   4: L:  61.4dB | R:  61.9dB | Diff:  -0.5dB | Avg:  61.7dB
   5: L:  62.5dB | R:  65.0dB | Diff:  -2.5dB | Avg:  63.8dB

Commands:
[s] Start/Stop recording
[l] Add label
[v] View recent data
[i] Change ESP32 IP
[q] Quit


...

Added label: energy_level = medium

Meeting Audio Recorder v1.0
ESP32: 192.168.0.110:81 - CONNECTED
Status: RECORDING
Recording Duration: 29.5 seconds
Samples Collected: 0
Session ID: meeting_20250624_134208

Current Labels:
   meeting_type: discussion
   energy_level: medium
Recent Audio Data:
   1: L:  60.5dB | R:  65.7dB | Diff:  -5.2dB | Avg:  63.1dB
   2: L:  63.2dB | R:  61.0dB | Diff:   2.2dB | Avg:  62.1dB
   3: L:  68.0dB | R:  66.3dB | Diff:   1.6dB | Avg:  67.1dB
   4: L:  70.0dB | R:  69.7dB | Diff:   0.3dB | Avg:  69.8dB
   5: L:  69.6dB | R:  69.8dB | Diff:  -0.2dB | Avg:  69.7dB

Commands:
[s] Start/Stop recording
[l] Add label
[v] View recent data
[i] Change ESP32 IP
[q] Quit



Enter command:  l



Available labels:
[1] Speaker count (1, 2, 3+)
[2] Meeting type (discussion, presentation, brainstorm, argument)
[3] Energy level (low, medium, high)
[4] Background noise (none, low, medium, high)
[5] Custom label


Select label type (1-5):  4
Background noise (none/low/medium/high):  low


Added label: background_noise = low

Meeting Audio Recorder v1.0
ESP32: 192.168.0.110:81 - CONNECTED
Status: RECORDING
Recording Duration: 36.7 seconds
Samples Collected: 0
Session ID: meeting_20250624_134208

Current Labels:
   meeting_type: discussion
   energy_level: medium
   background_noise: low
Recent Audio Data:
   1: L:  60.5dB | R:  65.7dB | Diff:  -5.2dB | Avg:  63.1dB
   2: L:  63.2dB | R:  61.0dB | Diff:   2.2dB | Avg:  62.1dB
   3: L:  68.0dB | R:  66.3dB | Diff:   1.6dB | Avg:  67.1dB
   4: L:  70.0dB | R:  69.7dB | Diff:   0.3dB | Avg:  69.8dB
   5: L:  69.6dB | R:  69.8dB | Diff:  -0.2dB | Avg:  69.7dB

Commands:
[s] Start/Stop recording
[l] Add label
[v] View recent data
[i] Change ESP32 IP
[q] Quit


In [ ]:
class MeetingDataCollector:
    def __init__(self, esp32_ip="192.168.1.100", port=81):
        self.esp32_ip = esp32_ip
        self.port = port
        self.ws_url = f"ws://{esp32_ip}:{port}"
        
        # Data storage
        self.audio_data = deque(maxlen=10000)  # Store last 10k samples
        self.session_data = []
        self.recording = False
        self.connected = False
        
        # Session metadata
        self.session_id = None
        self.session_labels = {}
        self.start_time = None
        
        # Create directories
        os.makedirs("recordings", exist_ok=True)
        os.makedirs("labels", exist_ok=True)
        
        # WebSocket setup
        self.ws = None
        self.running = True
        
        # Setup signal handler for graceful shutdown
        signal.signal(signal.SIGINT, self.signal_handler)
    
    def signal_handler(self, signum, frame):
        """Handle Ctrl+C gracefully"""
        print("\nShutting down gracefully...")
        self.stop_recording()
        self.running = False
        if self.ws:
            self.ws.close()
        sys.exit(0)
    
    def on_message(self, ws, message):
        """Handle incoming WebSocket messages"""
        try:
            data = json.loads(message)
            
            # Add local timestamp
            data['local_timestamp'] = time.time()
            data['local_datetime'] = datetime.now().isoformat()
            
            # Store data
            self.audio_data.append(data)
            
            # If recording, add to session data
            if self.recording:
                self.session_data.append(data)
                
        except json.JSONDecodeError:
            print(f"Error parsing message: {message}")
        except Exception as e:
            print(f"Error in on_message: {e}")
    
    def on_error(self, ws, error):
        """Handle WebSocket errors"""
        print(f"WebSocket error: {error}")
        self.connected = False
    
    def on_close(self, ws, close_status_code, close_msg):
        """Handle WebSocket close"""
        print("WebSocket connection closed")
        self.connected = False
    
    def on_open(self, ws):
        """Handle WebSocket open"""
        print("Connected to ESP32!")
        self.connected = True
    
    def connect_to_esp32(self):
        """Connect to ESP32 WebSocket"""
        print(f"Connecting to ESP32 at {self.ws_url}...")
        
        try:
            self.ws = websocket.WebSocketApp(
                self.ws_url,
                on_message=self.on_message,
                on_error=self.on_error,
                on_close=self.on_close,
                on_open=self.on_open
            )
            
            # Start WebSocket in separate thread
            self.ws_thread = threading.Thread(target=self.ws.run_forever)
            self.ws_thread.daemon = True
            self.ws_thread.start()
            
            # Wait for connection
            timeout = 10
            for i in range(timeout):
                if self.connected:
                    return True
                time.sleep(1)
                print(f"Waiting for connection... ({i+1}/{timeout})")
            
            print("Connection timeout!")
            return False
            
        except Exception as e:
            print(f"Connection failed: {e}")
            return False
    
    def start_recording(self):
        """Start a new recording session"""
        if self.recording:
            print("Already recording!")
            return
        
        # Generate session ID
        self.session_id = f"meeting_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        self.start_time = time.time()
        self.session_data = []
        self.session_labels = {
            'session_id': self.session_id,
            'start_time': datetime.now().isoformat(),
            'esp32_ip': self.esp32_ip
        }
        
        self.recording = True
        print(f"Recording started: {self.session_id}")
    
    def stop_recording(self):
        """Stop recording and save data"""
        if not self.recording:
            print("Not currently recording!")
            return
        
        self.recording = False
        duration = time.time() - self.start_time
        
        print(f"Recording stopped. Duration: {duration:.1f} seconds")
        print(f"Samples collected: {len(self.session_data)}")
        
        if len(self.session_data) > 0:
            self.save_session()
        else:
            print("No data to save!")
    
    def save_session(self):
        """Save session data to files"""
        if not self.session_data:
            print("No data to save!")
            return
        
        try:
            # Add final metadata
            self.session_labels['end_time'] = datetime.now().isoformat()
            self.session_labels['duration_seconds'] = time.time() - self.start_time
            self.session_labels['sample_count'] = len(self.session_data)
            
            # Save audio data as JSON
            audio_filename = f"recordings/{self.session_id}_audio.json"
            with open(audio_filename, 'w') as f:
                json.dump(self.session_data, f, indent=2)
            
            # Save labels as JSON
            labels_filename = f"labels/{self.session_id}_labels.json"
            with open(labels_filename, 'w') as f:
                json.dump(self.session_labels, f, indent=2)
            
            # Append to master CSV
            self.update_master_csv()
            
            print(f"Session saved:")
            print(f"   Audio: {audio_filename}")
            print(f"   Labels: {labels_filename}")
            
        except Exception as e:
            print(f"Error saving session: {e}")
    
    def update_master_csv(self):
        """Update master CSV with session summary"""
        csv_file = "labels/sessions_master.csv"
        
        # Create summary row
        summary = {
            'session_id': self.session_labels['session_id'],
            'start_time': self.session_labels['start_time'],
            'duration_seconds': self.session_labels['duration_seconds'],
            'sample_count': self.session_labels['sample_count'],
            'speaker_count': self.session_labels.get('speaker_count', 'unknown'),
            'meeting_type': self.session_labels.get('meeting_type', 'unknown'),
            'energy_level': self.session_labels.get('energy_level', 'unknown'),
            'background_noise': self.session_labels.get('background_noise', 'unknown'),
            'notes': self.session_labels.get('notes', '')
        }
        
        # Load existing CSV or create new one
        if os.path.exists(csv_file):
            df = pd.read_csv(csv_file)
            df = pd.concat([df, pd.DataFrame([summary])], ignore_index=True)
        else:
            df = pd.DataFrame([summary])
        
        df.to_csv(csv_file, index=False)
        print(f"Updated master CSV: {csv_file}")
    
    def add_label(self, key, value):
        """Add a label to current session"""
        if not self.recording:
            print("Start recording first!")
            return
        
        self.session_labels[key] = value
        print(f"Added label: {key} = {value}")
    
    def show_current_data(self):
        """Display current audio data"""
        if not self.audio_data:
            print("No data received yet...")
            return
        
        # Get last 5 samples
        recent_samples = list(self.audio_data)[-5:]
        
        print("Recent Audio Data:")
        for i, sample in enumerate(recent_samples, 1):
            left = sample.get('leftMic', 0)
            right = sample.get('rightMic', 0)
            diff = sample.get('difference', 0)
            avg = sample.get('averageLevel', 0)
            
            print(f"   {i}: L:{left:6.1f}dB | R:{right:6.1f}dB | Diff:{diff:6.1f}dB | Avg:{avg:6.1f}dB")
    
    def show_status(self):
        """Display current status"""
        status = "RECORDING" if self.recording else "STOPPED"
        connection = "CONNECTED" if self.connected else "DISCONNECTED"
        
        print(f"\n{'='*60}")
        print(f"Meeting Audio Recorder v1.0")
        print(f"ESP32: {self.esp32_ip}:{self.port} - {connection}")
        print(f"Status: {status}")
        
        if self.recording:
            duration = time.time() - self.start_time
            print(f"Recording Duration: {duration:.1f} seconds")
            print(f"Samples Collected: {len(self.session_data)}")
            print(f"Session ID: {self.session_id}")
            
            print(f"\nCurrent Labels:")
            for key, value in self.session_labels.items():
                if key not in ['session_id', 'start_time', 'esp32_ip']:
                    print(f"   {key}: {value}")
        
        print(f"{'='*60}")
    
    def interactive_menu(self):
        """Main interactive menu"""
        print("Starting Meeting Audio Data Collector...")
        
        # Connect to ESP32
        if not self.connect_to_esp32():
            print("Failed to connect to ESP32. Please check IP and ensure ESP32 is running.")
            return
        
        # Wait a moment for data to start flowing
        time.sleep(2)
        
        while self.running:
            try:
                self.show_status()
                self.show_current_data()
                
                print(f"\nCommands:")
                print(f"[s] Start/Stop recording")
                print(f"[l] Add label")
                print(f"[v] View recent data")
                print(f"[i] Change ESP32 IP")
                print(f"[q] Quit")
                
                choice = input("\nEnter command: ").strip().lower()
                
                if choice == 's':
                    if self.recording:
                        self.stop_recording()
                    else:
                        self.start_recording()
                
                elif choice == 'l':
                    if not self.recording:
                        print("Start recording first!")
                        continue
                    
                    print("\nAvailable labels:")
                    print("[1] Speaker count (1, 2, 3+)")
                    print("[2] Meeting type (discussion, presentation, brainstorm, argument)")
                    print("[3] Energy level (low, medium, high)")
                    print("[4] Background noise (none, low, medium, high)")
                    print("[5] Custom label")
                    
                    label_choice = input("Select label type (1-5): ").strip()
                    
                    if label_choice == '1':
                        value = input("Number of speakers (1, 2, 3+): ").strip()
                        self.add_label('speaker_count', value)
                    elif label_choice == '2':
                        value = input("Meeting type (discussion/presentation/brainstorm/argument): ").strip()
                        self.add_label('meeting_type', value)
                    elif label_choice == '3':
                        value = input("Energy level (low/medium/high): ").strip()
                        self.add_label('energy_level', value)
                    elif label_choice == '4':
                        value = input("Background noise (none/low/medium/high): ").strip()
                        self.add_label('background_noise', value)
                    elif label_choice == '5':
                        key = input("Label name: ").strip()
                        value = input("Label value: ").strip()
                        self.add_label(key, value)
                
                elif choice == 'v':
                    self.show_current_data()
                    input("Press Enter to continue...")
                
                elif choice == 'i':
                    new_ip = input(f"Current IP: {self.esp32_ip}. Enter new IP: ").strip()
                    if new_ip:
                        self.esp32_ip = new_ip
                        self.ws_url = f"ws://{new_ip}:{self.port}"
                        print(f"IP updated to {new_ip}. Restart to connect.")
                
                elif choice == 'q':
                    if self.recording:
                        print("Still recording! Stopping...")
                        self.stop_recording()
                    break
                
                else:
                    print("Invalid command!")
                
            except KeyboardInterrupt:
                break
            except Exception as e:
                print(f"Error in menu: {e}")
        
        print("Goodbye!")

def main():
    """Main function"""
    print("Meeting Audio Data Collector v1.0")
    print("="*50)
    
    # Get ESP32 IP
    default_ip = "192.168.1.100"
    esp32_ip = input(f"ESP32 IP address ({default_ip}): ").strip()
    if not esp32_ip:
        esp32_ip = default_ip
    
    # Create collector and start
    collector = MeetingDataCollector(esp32_ip)
    collector.interactive_menu()

if __name__ == "__main__":
    main()

Meeting Audio Data Collector v1.0


ESP32 IP address (192.168.1.100):  192.168.0.110


Starting Meeting Audio Data Collector...
Connecting to ESP32 at ws://192.168.0.110:81...
Connected to ESP32!
Waiting for connection... (1/10)

Meeting Audio Recorder v1.0
ESP32: 192.168.0.110:81 - CONNECTED
Status: STOPPED
Recent Audio Data:
   1: L:  75.9dB | R:  76.6dB | Diff:  -0.7dB | Avg:  76.2dB
   2: L:  68.7dB | R:  65.3dB | Diff:   3.3dB | Avg:  67.0dB
   3: L:  64.2dB | R:  65.6dB | Diff:  -1.4dB | Avg:  64.9dB
   4: L:  71.5dB | R:  64.1dB | Diff:   7.4dB | Avg:  67.8dB
   5: L:  59.0dB | R:  66.4dB | Diff:  -7.3dB | Avg:  62.7dB

Commands:
[s] Start/Stop recording
[l] Add label
[v] View recent data
[i] Change ESP32 IP
[q] Quit



Enter command:  s


Recording started: meeting_20250624_135319

Meeting Audio Recorder v1.0
ESP32: 192.168.0.110:81 - CONNECTED
Status: RECORDING
Recording Duration: 0.0 seconds
Samples Collected: 0
Session ID: meeting_20250624_135319

Current Labels:
Recent Audio Data:
   1: L:  59.0dB | R:  66.4dB | Diff:  -7.3dB | Avg:  62.7dB
   2: L:  63.5dB | R:  63.9dB | Diff:  -0.3dB | Avg:  63.7dB
   3: L:  60.9dB | R:  66.4dB | Diff:  -5.5dB | Avg:  63.6dB
   4: L:  61.5dB | R:  67.3dB | Diff:  -5.7dB | Avg:  64.4dB
   5: L:  69.1dB | R:  64.0dB | Diff:   5.1dB | Avg:  66.5dB

Commands:
[s] Start/Stop recording
[l] Add label
[v] View recent data
[i] Change ESP32 IP
[q] Quit


...


Available labels:
[1] Speaker count (1, 2, 3+)
[2] Meeting type (discussion, presentation, brainstorm, argument)
[3] Energy level (low, medium, high)
[4] Background noise (none, low, medium, high)
[5] Custom label


Select label type (1-5):  3
Energy level (low/medium/high):  high


Added label: energy_level = high

Meeting Audio Recorder v1.0
ESP32: 192.168.0.110:81 - CONNECTED
Status: RECORDING
Recording Duration: 74.3 seconds
Samples Collected: 0
Session ID: meeting_20250624_141351

Current Labels:
   speaker_count: 3+
   meeting_type: brainstorm
   energy_level: high
   background_noise: low
Recent Audio Data:
   1: L:  70.8dB | R:  64.0dB | Diff:   6.7dB | Avg:  67.4dB
   2: L:  73.6dB | R:  71.0dB | Diff:   2.7dB | Avg:  72.3dB
   3: L:  68.2dB | R:  63.2dB | Diff:   5.0dB | Avg:  65.7dB
   4: L:  65.6dB | R:  68.7dB | Diff:  -3.2dB | Avg:  67.1dB
   5: L:  64.0dB | R:  63.6dB | Diff:   0.4dB | Avg:  63.8dB

Commands:
[s] Start/Stop recording
[l] Add label
[v] View recent data
[i] Change ESP32 IP
[q] Quit



Enter command:  s


Recording stopped. Duration: 429.6 seconds
Samples collected: 0
No data to save!

Meeting Audio Recorder v1.0
ESP32: 192.168.0.110:81 - CONNECTED
Status: STOPPED
Recent Audio Data:
   1: L:  70.8dB | R:  64.0dB | Diff:   6.7dB | Avg:  67.4dB
   2: L:  73.6dB | R:  71.0dB | Diff:   2.7dB | Avg:  72.3dB
   3: L:  68.2dB | R:  63.2dB | Diff:   5.0dB | Avg:  65.7dB
   4: L:  65.6dB | R:  68.7dB | Diff:  -3.2dB | Avg:  67.1dB
   5: L:  64.0dB | R:  63.6dB | Diff:   0.4dB | Avg:  63.8dB

Commands:
[s] Start/Stop recording
[l] Add label
[v] View recent data
[i] Change ESP32 IP
[q] Quit
WebSocket error: Connection timed out
WebSocket connection closed


In [4]:
# Show me the contents of:
cat labels/sessions_master.csv

SyntaxError: invalid syntax (900107361.py, line 2)